In [ ]:
## Norine_crawling
import openpyxl as openpyxl
import pandas as pd
import requests
from lxml import etree
from urllib3 import disable_warnings
from urllib3.exceptions import InsecureRequestWarning

wb = openpyxl.Workbook()
sheet1 = wb.active
sheet1.append(['nid', 'name', 'mass', 'family', 'activaty', 'formula', 'synonyms', 'comment', 'smiles',
               'Amino_acid_composition', 'organism', 'pubmed', 'pubchem','chembl', 'pdb', 'uniprot'])
sheet2 = wb.create_sheet('similar')
sheet2.title = 'similar'
sheet2.append(['similar'])
disable_warnings(InsecureRequestWarning)
url_main = 'https://norine.univ-lille.fr/norine/result.jsp?ID='
headers = {
    'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/111.0.0.0 Safari/537.36 Edg/111.0.1661.54 "
}
# keyword = pd.read_excel('fileName.xlsx', dtype=object).loc[:, 'Column_Name']
keyword = ['NOR00043','NOR00283','NOR00284','NOR00286','NOR00249','NOR00315','NOR00050']# Test data

for i in range(len(keyword)):
    url = url_main + keyword[i]
    response = requests.get(url=url, headers=headers, verify=False)
    response.encoding = response.apparent_encoding
    tree = etree.HTML(response.text)
    tr1_list = tree.xpath('//*[@id="tabs-1"]/ul')
    name, mass, family, activaty, formula, synonyms, comment = '', '', '', '', '', '', ''
    pubchem, chembl, smiles, organism, pubmed, uniprot, pdb = '', '', '', '', '', '', ''
    Amino_acid_composition = ''
    for tr in tr1_list:
        name = ''.join(tree.xpath('//*[@id="center"]/h3/text()')).replace('\n', '').replace('\t', '')
        list1 = tr.xpath('./li')
        for li in list1:
            list_n = ''.join(li.xpath('.//text()')).strip()
            if 'mass' in list_n:
                mass = ''.join(li.xpath('./text()')).strip()
            if 'Family' in list_n:
                family = ''.join(li.xpath('./text()')).strip()
            if 'Activity' in list_n:
                activaty = ''.join(li.xpath('./text()')).strip()
            if 'Formula' in list_n:
                formula = ''.join(li.xpath('./text()')).strip()
            if 'Synonym' in list_n:
                synonyms = ''.join(li.xpath('./text()')).strip().rstrip(';')
            if 'Comment' in list_n:
                comment = ''.join(li.xpath('./text()')).strip()
    tr2_list = tree.xpath('//*[@id="tabs-2"]/ul')
    for tr in tr2_list:
        if 'Smiles' in ''.join(tr.xpath('./li[3]//text()')):
            smiles = ''.join(tr.xpath('./li[3]/text()')).strip()
        if 'Graph representation' in ''.join(tr.xpath('./li[4]//text()')):
            Amino_acid_composition = ''.join(tr.xpath('./li[4]/text()')).strip()
        elif 'Graph representation' in ''.join(tr.xpath('./li[6]//text()')):
            Amino_acid_composition = ''.join(tr.xpath('./li[6]/text()')).strip()
    organism = ''.join(tree.xpath('//*[@id="tabs-4"]/ul/li/i/text()')).strip()
    pubmed_list = tree.xpath('//*[@id="tabs-5"]/ul')
    for ul in pubmed_list:
        refer1_list = ul.xpath('./li/a')
        for ref in refer1_list:
            refer1 = ''.join(ref.xpath('./@href'))
            if 'PubMed' in refer1:
                pubmed += refer1.replace('http://www.ncbi.nlm.nih.gov/entrez/query.fcgi?cmd=search&db=PubMed&term=', '') + '##'
    refer_list = tree.xpath('//*[@id="tabs-6"]/a')
    for ref in refer_list:
        refer = ''.join(ref.xpath('./@href'))
        if 'uniprot' in refer:
            uniprot += refer.replace('http://www.uniprot.org/uniprot/', '') + '##'
        if 'pubchem' in refer:
            pubchem += refer.replace('http://pubchem.ncbi.nlm.nih.gov/summary/summary.cgi?cid=', '') + '##'
        if 'chembldb' in refer:
            chembl += refer.replace('https://www.ebi.ac.uk/chembldb/compound/inspect/', '') + '##'
        if 'structureId' in refer:
            pdb += refer.replace('http://www.rcsb.org/pdb/explore/explore.do?structureId=', '') + '##'
    xx_info = [keyword[i], name, mass, family, activaty, formula, synonyms, comment, smiles, Amino_acid_composition, organism, pubmed.rstrip('##'), pubchem.rstrip('##'), chembl.rstrip('##'), pdb.rstrip('##'), uniprot.rstrip('##')]
    print(xx_info)
    sheet1.append(xx_info)
wb.save('norine_Info.xlsx')
